# Introdução

Projeto final da cadeira IF807 que tem como o objetivo a implementação de uma técnica dentro do tema de Fairness em sistemas de recomendação. A técnica é baseada em um artigo que pode ser encontrado nesse link: https://dl.acm.org/doi/10.1145/3292500.3330691

Equipe:



O objetivo é implementar um sistema de re-ranqueamento que consideram o Fainess, para que então ocorra menos viéses na hora de gerar listas de ranking em algoritmos de recomendação. Com essa proposta, esse viés é quantificado e mitigado, para um determinado subconjunto de dados, esses algoritmos podem gerar uma distribuição desejada dos resultados mais bem classificados, mantendo assim a paridade demográfica ou a igualdade de oportunidades conforme necessário. 

As métricas são definidas para avaliar o nível de Fairness então alcançado e são mostrador resultados obtidos através de dados sinteticamente gerados.

# Implementação

### Bibliotecas

In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

### Medidas para avaliação de viés

O artigo lista algumas medidas que podem ser utilizadas para obter o valor de viés de cada um dos algoritmos de recomendação, eles são definidos como o seguinte:

##### Medidas baseada nos top-k resultados

##### Skew@k

O Desvio dos top-k resultados rankeados da proporção desejada é quantificada usando uma métrica skew (assimetria). A função log retorna um valor negativo de assimetria para under representation ou positiva para over representation do atributo que está sendo avaliado. 

In [2]:
def proportion_calc(df,k,ai):
    count=0
    for i in range(k):  
        if (df[i] == ai): 
            count = count+1
    return (count/k)    

def skew_func(df,p,k,ai):
    s = math.log((proportion_calc(df,k,ai)+1)/(p[ai]+1)) 
    return s

A métrica acima tem alguns problemas, primeiro o fato de que é definida apenas sobre o valor de um atributo e precisa ser estendida para aglobar vários, e também, o valor computado depende altamente do valor k (top valores) e então uma métrica acumulativa e compreensiva é requerida. Para resolver esses problemas, são definidas métricas MinSkew e MaxSkew que quantificam a pior desvantagem e maior vantagem para um candidato.

##### MinSkew@k

In [3]:
def min_skew(df,p,k):
    min_skew_var=float('inf')
    for x in range(len(p)):
        m=skew_func(list(df),p,k,x)
        if m < min_skew_var:
            min_skew_var = m
    return min_skew_var

##### MaxSkew@k

In [5]:
def max_skew(df,p,k):
    max_skew_var=-float('inf')
    for x in range(len(p)):
        m=skew_func(list(df),p,k,x)
        if m > max_skew_var:
            max_skew_var = m
    return max_skew_var

Para o segundo problema citado acima, métricas de ranqueamento são definidas e listadas abaixo.

### Medidas de ranqueamento

##### NDKL

A divergência Kullback-Leibler (KL) é uma medida não negativa no qual o valor maior denota uma grande divergência entre as distribuições, ela é implementada como uma medida acumulativa envolvendo uma média baseada em pesos de Skew@i sobre todos os valores de atributo.

Um viés de magnitude igual mas em direções opostas não pode ser distinguido por essa métrica, por isso ele não indica qual atributo está sendo tratado de forma injusta.

In [6]:
def kld_func(D1,D2):
    a = np.asarray(D1, dtype=np.float)
    b = np.asarray(D2, dtype=np.float)

    return np.sum(np.where(a != 0 , a * np.log((a +0.00001)/(b+0.00001)), 0))

def ndkl_func(df,p):
    Z = np.sum(1/(np.log2(np.arange(1,len(df)+1)+1)))
    total=0

    for i in range(1,len(df)+1): 
        value=df[:i].value_counts(normalize = True)
        value=value.to_dict()
        D1=[]
        for i in range(len(p)):
            if i in value.keys():
                D1.append(value[i])
            else:
                D1.append(0)
        total=total+(1/math.log2(i+1)) * kld_func(D1,p)

    return (1/Z)*total

##### NDCG

Ganho de desconto acumulativo normalizado, o ganho cumulativo captura que os resultados muito relevantes são mais úteis do que os resultados parcialmente relevantes, que, por sua vez, são mais úteis do que os resultados irrelevantes. Dependendo da noção de relevância, que para o caso do teste é o score de cada candidato.

In [8]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0

def ndcg_at_k(df, k, method=0):
    r=list(df)
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max